In [1]:
import os
from dotenv import dotenv_values

env_path = os.path.join(os.getcwd(), '../../..', '.env')
env_dict = dotenv_values(env_path)

os.environ['CODEGEEX_CHAT_PRO_TOKEN'] = env_dict['CODEGEEX_CHAT_PRO_TOKEN']

In [ ]:
import json
import sys
import os

sys.path.append('/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness')

from bigcode_eval.tasks.code_to_code.codegeex_chat_pro import codegeex_chat_pro_translate_and_postprocess

prompt_version='v1'

translated_prompts_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/humaneval-x/humaneval_python_java_prompts_{prompt_version}.json'

limit_start = 0
limit = len(json.load(open(translated_prompts_path, "r")))
model = 'codegeex-chat-pro'

save_translated_codes_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/{model}-humaneval_python_java_prompts_{prompt_version}-translations-{limit_start}-{limit}.json'

codegeex_chat_pro_translate_and_postprocess(
    translated_prompts_path, save_translated_codes_path, limit_start, limit, parallel=False)

In [10]:
import json
import sys
import os

sys.path.append('/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness')

from bigcode_eval.tasks.code_to_code.codegeex_chat_pro import codegeex_chat_pro_translate_and_postprocess

prompt_version='v1.1'

translated_prompts_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/humaneval-x/humaneval_python_java_prompts_{prompt_version}.json'

limit_start = 0
limit = len(json.load(open(translated_prompts_path, "r")))
model = 'codegeex-chat-pro'

save_translated_codes_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/{model}-humaneval_python_java_prompts_{prompt_version}-translations-{limit_start}-{limit}.json'

codegeex_chat_pro_translate_and_postprocess(
    translated_prompts_path, save_translated_codes_path, limit_start, limit, parallel=True, max_workers=5)


Using parallel processing with #5 workers
Translated 164 prompts in parallel
saved #164 translations to /Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/codegeex-chat-pro-humaneval_python_java_prompts_v1.1-translations-0-164.json


In [5]:
import json
import sys
import os

sys.path.append('/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness')

from bigcode_eval.tasks.code_to_code.codegeex_chat_pro import codegeex_chat_pro_translate_and_postprocess

prompt_version='v2'

translated_prompts_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/humaneval-x/humaneval_python_java_prompts_{prompt_version}.json'

limit_start = 0
limit = len(json.load(open(translated_prompts_path, "r")))
model = 'codegeex-chat-pro'

save_translated_codes_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/{model}-humaneval_python_java_prompts_{prompt_version}-translations-{limit_start}-{limit}.json'

codegeex_chat_pro_translate_and_postprocess(
    translated_prompts_path, save_translated_codes_path, limit_start, limit, parallel=False)

Translating Prompts: 100%|██████████| 164/164 [09:28<00:00,  3.47s/prompt]

Translated 164 prompts in serial
saved #164 translations to /Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/codegeex-chat-pro-humaneval_python_java_prompts_v2-translations-0-164.json


In [ ]:
prompt = """
Convert Python to Java code step-by-step starting with the predefined template:
Java:
```
import java.lang.*;
import java.util.*;

class Solution {
    public List <String> separateParenGroups(String paren_string) {
```

Python:
```from typing import List

def separate_paren_groups(paren_string: str) -> List[str]:
    result = []
    current_string = []
    current_depth = 0

    for c in paren_string:
        if c == '(':
            current_depth += 1
            current_string.append(c)
        elif c == ')':
            current_depth -= 1
            current_string.append(c)

            if current_depth == 0:
                result.append(''.join(current_string))
                current_string.clear()

    return result
```
"""

import json
import sys
import os

sys.path.append('/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness')

from bigcode_eval.tasks.code_to_code.codegeex_chat_pro import codegeex_chat_pro_translate

print(codegeex_chat_pro_translate(prompt))

In [ ]:
import json
import sys
import os

sys.path.append('/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness')

from bigcode_eval.tasks.code_to_code.codegeex_chat_pro import codegeex_chat_pro_translate_and_postprocess

prompt_version='v3'

translated_prompts_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/humaneval-x/humaneval_python_java_prompts_{prompt_version}.json'

limit_start = 0
limit = len(json.load(open(translated_prompts_path, "r")))
model = 'codegeex-chat-pro'

save_translated_codes_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/{model}-humaneval_python_java_prompts_{prompt_version}-translations-{limit_start}-{limit}.json'

codegeex_chat_pro_translate_and_postprocess(
    translated_prompts_path, save_translated_codes_path, limit_start, limit, parallel=True)


In [6]:
import json
import sys
import os

sys.path.append('/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness')

from bigcode_eval.tasks.code_to_code.codegeex_chat_pro import codegeex_chat_pro_translate_and_postprocess

prompt_version='v4'

translated_prompts_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/humaneval-x/humaneval_python_java_prompts_{prompt_version}.json'

limit_start = 0
limit = len(json.load(open(translated_prompts_path, "r")))
model = 'codegeex-chat-pro'

save_translated_codes_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/{model}-humaneval_python_java_prompts_{prompt_version}-translations-{limit_start}-{limit}.json'

codegeex_chat_pro_translate_and_postprocess(
    translated_prompts_path, save_translated_codes_path, limit_start, limit, parallel=True)


Using parallel processing with #5 workers
Translated 164 prompts in parallel
saved #164 translations to /Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/codegeex-chat-pro-humaneval_python_java_prompts_v4-translations-0-164.json


In [8]:
import json
import sys
import os

sys.path.append('/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness')

from bigcode_eval.tasks.code_to_code.codegeex_chat_pro import codegeex_chat_pro_translate_and_postprocess

prompt_version='v5'

translated_prompts_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/humaneval-x/humaneval_python_java_prompts_{prompt_version}.json'

limit_start = 0
limit = len(json.load(open(translated_prompts_path, "r")))
model = 'codegeex-chat-pro'

save_translated_codes_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/{model}-humaneval_python_java_prompts_{prompt_version}-translations-{limit_start}-{limit}.json'

codegeex_chat_pro_translate_and_postprocess(
    translated_prompts_path, save_translated_codes_path, limit_start, limit, parallel=True)


Using parallel processing with #5 workers
Translated 164 prompts in parallel
saved #164 translations to /Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/codegeex-chat-pro-humaneval_python_java_prompts_v5-translations-0-164.json


In [9]:
import json
import sys
import os

sys.path.append('/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness')

from bigcode_eval.tasks.code_to_code.codegeex_chat_pro import codegeex_chat_pro_translate_and_postprocess

prompt_version='v6'

translated_prompts_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/datasets/humaneval-x/humaneval_python_java_prompts_{prompt_version}.json'

limit_start = 0
limit = len(json.load(open(translated_prompts_path, "r")))
model = 'codegeex-chat-pro'

save_translated_codes_path = f'/Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/{model}-humaneval_python_java_prompts_{prompt_version}-translations-{limit_start}-{limit}.json'

codegeex_chat_pro_translate_and_postprocess(
    translated_prompts_path, save_translated_codes_path, limit_start, limit, parallel=True, max_workers=5)


Using parallel processing with #5 workers
Translated 164 prompts in parallel
saved #164 translations to /Users/phuonglvh/projects/2170558-thesis-automatic-code-generation-using-machine-learning/bigcode-evaluation-harness/benchmark/codegeex-chat-pro/humaneval-x/codegeex-chat-pro-humaneval_python_java_prompts_v6-translations-0-164.json
